# Obtain valid set of reactions 
Required Python code to load pertinent modules.

In [ ]:
from thermoengine import phases
from thermoengine import model
from thermoengine import chem
import thermoengine as thermo

import numpy as np
import pandas as pd

import pickle

In [ ]:
data_dir="/Users/jennaadams/Documents/projects/garnet-soln/data/"

In [ ]:
modelDB = model.Database()

# Quick aside on calculating molar endmember compositions

In [ ]:
phs = modelDB.get_phase('Grt')
#print('phs = ', phs.endmember_names, ' --> ', phs.endmember_formulas) #does not function properly

In [ ]:
mol_oxide_alm = chem.format_mol_oxide_comp({'FeO':3, 'Al2O3':1, 'SiO2':3})
mol_oxide_grs = chem.format_mol_oxide_comp({'CaO':3, 'Al2O3':1, 'SiO2':3})
mol_oxide_pyr = chem.format_mol_oxide_comp({'MgO':3, 'Al2O3':1, 'SiO2':3})

mol_oxide_mix =  chem.format_mol_oxide_comp({'FeO':.9, 'CaO':1, 'MgO':1.1, 
                                             'Al2O3':1, 'SiO2':3})

In [ ]:
mol_oxide_alm

In [ ]:
print('Endmember inferences:')
print('alm: ', phs.calc_endmember_comp(mol_oxide_alm)[0])
print('grs: ', phs.calc_endmember_comp(mol_oxide_grs)[0])
print('pyr: ', phs.calc_endmember_comp(mol_oxide_pyr)[0])
print('mix: ', phs.calc_endmember_comp(mol_oxide_mix, method='intrinsic')[0])

## We already have an update to calculating endmember comp in master (that we need to pull in)
* To use it, just add the keyword argument to calc_endmember_comp:
    * method='intrinsic'

In [ ]:
rxn_data=pd.read_excel(data_dir+'grt_bearing_expts.xls',sheetname=None)

In [ ]:
rxn_data['phase_symbols']

### Find relevant phases according to data

In [ ]:
#delete this?
#rel_phases = rxn_data['phase_symbols']['phase_symbol'].tolist()

#modelDB.phase_symbols
#mask_phases = [iphase in modelDB.phase_symbols for iphase in rel_phases]
#rel_phases[mask_phases]

In [ ]:
#relevant_phases = chem.get_relevant_phases(rxn_data)

In [ ]:
#relevant_phases

In [ ]:
phase_names = phases.PurePhase

In [ ]:
phase_names

In [ ]:
#props = modelDB.phase_attributes['props']

In [ ]:
phases_present=rxn_data['phase_symbols']
relevant_phases = rxn_data['phase_symbols']['phase_symbol'].tolist()
#endmember_names = phases.PurePhase(relevant_phases)

### Retrieve singular value decomposition matrix for all reactions:
➜ output is a matrix of valid, linearly independent reactions that minimize variance  and maximizes orthogonality of each vector against another.

➜ Input a tolerance, defined as...

In [ ]:
rxn_svd, rxn_svd_props = chem.calc_reaction_svd(relevant_phases, TOLsvd=1e-4)

In [ ]:
rxn_svd

In [ ]:
rxn_svd_props['all_endmember_id']

In [ ]:
rxn_svd_props['all_endmember_name']

In [ ]:
Nbasis=len(rxn_svd)

### Get set of filtered reactions
➜ output is matrix of valid, lineraly independent reactions filtered based on given input variables described below

➜ input

    rxn_svd: matrix from above

    Ndraw: 

    ortho_scale: ratio that dictates level of rxn complexity (orthogonality/simplicity)

    Nbasis:

    TOL: rxn_complexity
 
note: the following operation make take ~10 minutes

In [ ]:
wtcoefs, costs, rxn_coefs, wtcoefs_ortho = chem.get_rxns(rxn_svd, Ndraw=2, ortho_scale=15, Nbasis=Nbasis, TOL=1e-10)

In [ ]:
rxns_file = '/Users/jennaadams/Documents/projects/ThermoEngine/Notebooks/Solutions/data/rxns_file.pkl'
with open(rxns_file, 'wb') as handle:
    pickle.dump(rxn_coefs, handle)
    
with open (rxns_file, 'rb') as handle:
    rxns_file_open = pickle.load(handle)
    
rxns_file_open

## Reaction Visiualization

In [ ]:
import matplotlib.pyplot as plt
%matplotlib notebook

### Visualizing rxn_svd

In [ ]:
rxn_max = np.max(rxn_svd)

plt.figure()
plt.imshow(rxn_svd,cmap='seismic')
plt.clim(-rxn_max,+rxn_max)
plt.colorbar()

plt.xlabel('Endmember ID#')
plt.ylabel('Balanced Rxn ID#')

### All reactions after filtering rxn_svd

In [ ]:
plt.figure()
plt.imshow(rxn_coefs,cmap='seismic')
plt.clim(-rxn_max, +rxn_max)
plt.colorbar()

plt.xlabel('Endmember ID#')
plt.ylabel('Balanced Rxn ID#')

### Reaction correlation

In [ ]:
ortho_project =np.dot(rxn_coefs, rxn_coefs.T) 

plt.figure()
plt.imshow(ortho_project, cmap='seismic')
plt.colorbar()
plt.clim(-1,1)

plt.xlabel('Basic Rxn ID#')
plt.ylabel('Basic Rxn ID#')
plt.title('Basic Rxn Correlation')

### Subset of new reactions 

In [ ]:
plt.figure()
plt.plot(rxn_coefs[::10].T, '-')
plt.xlabel('')
plt.ylabel('rxn')
plt.legend(['0','10','20','30'])

### Create final reaction matrix

In [ ]:
unique_phase_symbols=chem.get_phase_symbols(rxn_data)
rxn_svd_props = chem.calc_reaction_svd(unique_phase_symbols, TOLsvd=1e-4)
rxn_svd = rxn_svd_props['rxn_svd']
Nbasis=len(rxn_svd)
rxn_endmember_name = rxn_svd_props['all_endmember_name']
rxn_phase_symbols = rxn_svd_props['all_phase_symbol']



In [ ]:
rxn_svd_props.keys()

In [ ]:
rxn_svd_props['all_phase_ind']

In [ ]:
wtcoefs, costs, rxn_coefs, wtcoefs_ortho = chem.get_rxns(rxn_svd, Ndraw=2, ortho_scale=15, Nbasis=Nbasis, TOL=1e-10)

In [ ]:
(np.place(rxn_coefs, abs(rxn_coefs)< 1e-2, 0))

In [ ]:
endmember_ids = rxn_svd_props['all_endmember_id']

In [ ]:
len(rxn_phase_symbols)

In [ ]:
len(endmember_ids)

In [ ]:
len(rxn_endmember_name)

In [ ]:
rxns = []
for irxn_coefs in rxn_coefs:
    irxn = modelDB.get_rxn(rxn_phase_symbols, endmember_ids, irxn_coefs)
    
    rxns.append(irxn)    

In [ ]:
phases = []

for irxn in rxns:
    phases.extend(irxn.phases)
    
phases = np.unique(phases)

In [ ]:
rxn_phase_symbols

In [ ]:
phases

In [ ]:
rxn_coefs[0]